In [2]:

import gzip
import math
import numpy as  np
import random
import sklearn
import string
from collections import defaultdict
import json
def parseData(file):
    for l in open(file,'r'):
        yield json.loads(l)

In [3]:


dataset2 = []
dataset2 = list(parseData('renttherunway_final_data.json'))

dataset2[0]

FileNotFoundError: [Errno 2] No such file or directory: 'renttherunway_final_data.json'

In [ ]:
fittingPerUser = defaultdict(list)
fittingPerItem = defaultdict(list)

for entry in dataset2:
    item_id = entry['item_id']
    user_id = entry['user_id']
    fit = entry['fit']
    fittingPerUser[user_id].append((item_id,fit))
    fittingPerItem[item_id].append((user_id,fit))

print('Size of the dataset :', len(dataset2))
print('Size of the user base: ', len(fittingPerUser))
print('Size of the item base: ', len(fittingPerItem))

Size of the dataset : 82790
Size of the user base:  105571
Size of the item base:  5850


In [63]:
lengths = []
for u in fittingPerUser:
    lengths.append(len(fittingPerUser[u]))
print("average fitting per user :", np.array(lengths).mean())
print('number of unique users: ', len(lengths))
lengths = []
for i in fittingPerItem:
    lengths.append(len(fittingPerItem[i]))

print("average fitting per item :", np.array(lengths).mean())
print('number of unique items: ', len(lengths))

print('-----------------------------')
print("Remove users with too few reviews")
print('-----------------------------')

lengths = []
for u in fittingPerUser:
    if len(fittingPerUser[u])>2:
        lengths.append(len(fittingPerUser[u]))
print("average fitting per user post filterng:", np.array(lengths).mean())
print("total number of users post filtering :", len(lengths))


average fitting per user : 1.8238341968911918
average fitting per item : 32.91350427350427


In [66]:
lengths = []
total_reviews = 0
for u in fittingPerUser:
    if len(fittingPerUser[u])>2:
        lengths.append(len(fittingPerUser[u]))
        total_reviews+= len(fittingPerUser[u])
print("average fitting per user after filtering:", np.array(lengths).mean())
print(len(lengths))
print(total_reviews)
lengths = []
for i in fittingPerItem:
    lengths.append(len(fittingPerItem[i]))

print("average fitting per item :", np.array(lengths).mean())

average fitting per user : 5.3576835730507195
15852
84930
average fitting per item : 32.91350427350427


In [33]:
users = list(ratingsPerUser.keys())
items = list(ratingsPerItem.keys())
userIDs = {}
itemIDs = {}
for u,b,r in allRatings:
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not b in itemIDs: itemIDs[b] = len(itemIDs)
    
nTrain = len(ratingsTrain)
nTest = len(ratingsValid) 
interactionsTrain = ratingsTrain
interactionsTest = ratingsValid

# %%
import tensorflow as tf
optimizer = tf.keras.optimizers.Adam(0.0050)
mean_rating = np.array([i[2] for i in allRatings]).mean()
print(mean_rating)

class LatentFactorModel(tf.keras.Model):
    def __init__(self, mu, K, lamb1, lamb2):
        super(LatentFactorModel, self).__init__()
        # Initialize to average
        self.alpha = tf.Variable(mu,dtype='float32')
        # Initialize to small random values
        self.betaU = tf.Variable(tf.random.normal([len(userIDs)],stddev=0.001),dtype='float32')
        self.betaI = tf.Variable(tf.random.normal([len(itemIDs)],stddev=0.001),dtype='float32')
        self.gammaU = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.001),dtype='float32')
        self.gammaI = tf.Variable(tf.random.normal([len(itemIDs),K],stddev=0.001),dtype='float32')
        self.lamb1 = lamb1
        self.lamb2 = lamb2

    # Prediction for a single instance (useful for evaluation)
    def predict(self, u, i):
        p = self.alpha + self.betaU[u] + self.betaI[i] +\
            tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p

    # Regularizer
    def reg(self):
        return self.lamb1 * (tf.reduce_sum(self.betaU**2) +\
                            tf.reduce_sum(self.betaI**2)) +\
                            self.lamb2*(tf.reduce_sum(self.gammaU**2) +\
                            tf.reduce_sum(self.gammaI**2))
    
    # Prediction for a sample of instances
    def predictSample(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_u = tf.nn.embedding_lookup(self.betaU, u)
        
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        pred = self.alpha + beta_u + beta_i +\
               tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return pred
    
    # Loss
    def call(self, sampleU, sampleI, sampleR):
        pred = self.predictSample(sampleU, sampleI)
        r = tf.convert_to_tensor(sampleR, dtype=tf.float32)
        return tf.nn.l2_loss(pred - r) / len(sampleR)

# %%
def trainingStep(model, interactions):
    Nsamples =65000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleR = [], [], []
        for _ in range(Nsamples):
            u,i,r = random.choice(interactions)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleR.append(r)

        loss = model(sampleU,sampleI,sampleR)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

# %%
modelLFM = LatentFactorModel(mean_rating, 5, 0.000008,0.000038)
for i in range(210):
    obj = trainingStep(modelLFM, allRatings)
    if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))



1378